## 1. SQLContext

Importar la librería de pyspark y crea una SparkSession.

In [30]:
# Importar librerías y dependencias
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
# SparkSession
spark = SparkSession.builder.appName("walmart").getOrCreate()

22/09/24 16:51:09 WARN Utils: Your hostname, gmachin resolves to a loopback address: 127.0.1.1; using 192.168.0.19 instead (on interface wlp3s0)
22/09/24 16:51:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/24 16:51:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

## 2. Cargar datos

Carga los datos de Walmart (.csv) contenidos en el directorio ficheros:

In [10]:
df = spark.read.csv("./ficheros/walmart_stock.csv", header=True, inferSchema=True)

df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



Analiza que tipo de datos son, y sus valores:

In [11]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [15]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

## 3. Ejercicios

Crea una nueva columna denominada HV Ratio que sea el ratio de High vs. Volume calculado por día:

In [22]:
df = df.withColumn("HV Ratio", df["High"]/df["Volume"])

df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+--------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|            HV Ratio|
+----------+------------------+---------+---------+------------------+--------+------------------+--------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|7.367338463826307E-6|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|8.915604778943901E-6|
+----------+------------------+---------

¿Qué día se cotizo más alto?

In [33]:
df.orderBy(col("High").desc()).show(1)

+----------+---------+---------+-----+---------+-------+---------+--------------------+
|      Date|     Open|     High|  Low|    Close| Volume|Adj Close|            HV Ratio|
+----------+---------+---------+-----+---------+-------+---------+--------------------+
|2015-01-13|90.800003|90.970001|88.93|89.309998|8215400|83.825448|1.107310672639189...|
+----------+---------+---------+-----+---------+-------+---------+--------------------+
only showing top 1 row



In [37]:
# Registrar el DtaFrame como una vista temporal
df.createOrReplaceTempView("walmart")

sql_df = spark.sql(
    """
    select *
    from walmart
    order by High desc
    limit 1
    """
)

sql_df.show()

+----------+---------+---------+-----+---------+-------+---------+--------------------+
|      Date|     Open|     High|  Low|    Close| Volume|Adj Close|            HV Ratio|
+----------+---------+---------+-----+---------+-------+---------+--------------------+
|2015-01-13|90.800003|90.970001|88.93|89.309998|8215400|83.825448|1.107310672639189...|
+----------+---------+---------+-----+---------+-------+---------+--------------------+

